In [25]:
import yaml
import os.path as osp
import os

import numpy as np
import matplotlib.pyplot as plt
import igl
from tqdm import tqdm

import torch

from preprocess import mesh_sampling_method
from dataset import MeshData
from models import VAE_coma, GraphPredictor

In [26]:
# read into the config file
config_path = 'config/general_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# set the device, we can just assume we are using single GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = config["dataset"]
template = config["template"]
data_dir = osp.join('data', dataset)
template_fp = osp.join('template', template)

# get the up/down sampling matrix
ds_factor = config["ds_factor"]
edge_index_list, down_transform_list, up_transform_list = mesh_sampling_method(data_fp=data_dir,
                                                                                template_fp=template_fp,
                                                                                ds_factors=ds_factor,
                                                                                device=device)

# create the model
in_channels = config["model"]["in_channels"]
out_channels = config["model"]["out_channels"]
latent_channels = config["model"]["latent_channels"]
K = config["model"]["K"]

# get the mean and std of the CAESAR dataset(Traing set)
CAESAR_meshdata = MeshData(root=data_dir, template_fp=template_fp)
# of shape (10002, 3). is torch.Tensor
caesar_mean = CAESAR_meshdata.mean
caesar_std = CAESAR_meshdata.std

Normalizing the training and testing dataset...
Normalization Done!


## Load the predictor and the CoMA

In [27]:
predictor = GraphPredictor(in_channels = in_channels, out_channels = out_channels,
                        edge_index = edge_index_list, down_transform = down_transform_list, K=K).to(device)
predictor.load_state_dict(torch.load("predictor_network/predictor.pth"))

height_mean = 1716.4373
height_std = 107.98842

arm_length_mean = 612.611
arm_length_std = 45.986

crotch_height_mean = 773.540
crotch_height_std = 55.731

chest_mean = 996.745
chest_std = 124.099

hip_mean = 1050.170
hip_std = 113.026

waist_mean = 848.005
waist_std = 144.338

In [28]:
model = VAE_coma(in_channels = in_channels,
                out_channels = out_channels,
                latent_channels = latent_channels,
                edge_index = edge_index_list,
                down_transform = down_transform_list,
                up_transform = up_transform_list,
                K=K).to(device)


# model_path = "out/vae_coma/trainer12/20231220-123538/model.pth" # our method
# model_path = "out/vae_coma/trainer13/20240119-003904/model.pth" # our method without Laplacian loss
model_path = "out/vae_coma/trainer14/20240119-090527/model.pth" # our method without AWL
# model_path = "out/vae_coma/trainer15/20240119-193650/model.pth" # our method without disentanglement

model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

## Define the decoder function

In [29]:
def decode_func(x, model):
    num_layers = len(model.de_layers)
    num_deblocks = num_layers - 2
    for i, layer in enumerate(model.de_layers):
        if i == 0:
            x = layer(x)
            x = x.view(-1, model.num_vert, model.out_channels[-1])
        elif i != num_layers - 1:
            x = layer(x, model.edge_index[num_deblocks - i],
                        model.up_transform[num_deblocks - i])
        else:
            # last layer
            x = layer(x, model.edge_index[0])
    return x 

## Define the computation process

In [30]:
# TODO: We need further modification to this function
def get_feature_vals(model, sample_vals: np.ndarray, mean: np.ndarray, std: np.ndarray, predictor):
    
    """_summary_
        load the model from model_path
        sample points from the latent space
        return the latent values and the corresponding height values and weight values

    Args:
        model: the generative model
        sample_vals (np.ndarray): of shape (n_samples, ). We would sample n_samples from the latent space. And use the same vals for all 8 dimensions.
    """

    # Sample from the latent space
    # create a tensor of shape (n_samples, 8, 6)
    feature_vals = np.zeros((sample_vals.shape[0], 8, 6))

    # for each latent dimension
    for i in tqdm(range(8)):
        latent_val = torch.zeros((sample_vals.shape[0], 8))

        latent_val[:, i] = torch.Tensor(sample_vals)
        latent_val = latent_val.to(device) 
        
        # decode the latent values
        # of shape (n_samples, 10002, 3)
        v = decode_func(latent_val, model)
        
        # get the weight and height of the mesh
        features = predictor(v)

        # features is of shape (n_samples, 6)
        # mean is of shape (6, ), std is of shape (6, )
        features = features.cpu().detach().numpy()
        features = features * std + mean
       
        feature_vals[:, i, :] = features
        
    return feature_vals

## Define the plot function

In [31]:
# TODO: We need furthur modification to the plot_latent function
def plot_latent(filename: str, sample_vals: np.ndarray, feature_vals: np.ndarray):
    """_summary_
        declare a plot of 6 subplots, 6 rows and 1 column
        each subplot is a dot plot of the corresponding latent dimension
        the x-axis is the latent value, the y-axis is the height
        the figure size is tight to the subplots
    Args:
        filename (str): the file name to save the plot.
        sample_vals (np.ndarray): of shape (n_samples, ). We would sample n_samples from the latent space. And use the same vals for all 8 dimensions.
        feature_vals (np.ndarray): of shape (n_samples, 8, 6). 8 is the number of latent dimensions. 6 is the number of features.
    """

    # Ensure that the dimensions match up correctly
    if len(sample_vals) != feature_vals.shape[0] or feature_vals.shape[1] != 8 or feature_vals.shape[2] != 6:
        raise ValueError("Input dimensions are mismatched!")

    labels = ["Dim1", "Dim2", "Dim3", "Dim4", "Dim5", "Dim6", "Dim7", "Dim8"]
    lines = []
    fig, axs = plt.subplots(6, 1, figsize=(5, 20), sharex=True)

    for i, ax in enumerate(axs.flat):
        
        for j in range(8):
            line, = ax.plot(sample_vals, feature_vals[:, j, i], label=labels[j])
        
            if i == 0:
                lines.append(line)
        ax.set_xlim(-3, 3)


    # first create a dummy legend, so fig.tight_layout() makes enough space
    axs[0].legend(handles=lines, labels = labels, ncol=4,
                    bbox_to_anchor=(0.5, 1.0), loc='lower center')
    fig.tight_layout()
    # # now create the real legend; if fig.tight_layout() were called on this,
    # #  it would create a large empty space between the columns of subplots
    # #  as it wants the legend to belong to only one of the subplots
    # axs[0].legend(handles=lines, labels = labels, ncol=4,
    #                 bbox_to_anchor=(1.03, 1.12), loc='lower center', fontsize=8)

    # Save the figure to the given filename
    plt.savefig(filename)
    
    # Close the figure
    plt.close(fig)

In [32]:
sample_vals = np.linspace(-3, 3, 50)

mean = np.array([height_mean, arm_length_mean, crotch_height_mean, chest_mean, hip_mean, waist_mean])
std = np.array([height_std, arm_length_std, crotch_height_std, chest_std, hip_std, waist_std])

feature_vals = get_feature_vals(model, sample_vals, mean, std, predictor)

plot_latent("result/together/wo_awl_features.pdf", sample_vals, feature_vals)



100%|██████████| 8/8 [00:00<00:00, 22.38it/s]


## generate mesh samples

In [33]:
# read into the template mesh
v, f = igl.read_triangle_mesh(template_fp)


def generate_a_mesh(model, latent_vector: np.ndarray, mean, std, fp, name):
    mean = mean.to(device)
    std = std.to(device)
    
    latent_vector = torch.Tensor(latent_vector).cuda()
    latent_vector = latent_vector.reshape(1, -1)
    
    v = decode_func(latent_vector, model)
    v = v.detach()
    v = v * std + mean
    v = v.cpu().numpy()
    v = v.reshape(-1, 3)
    
    # find the lowest vertex
    z_min = v[:, 2].min()
    # move the lowest vertex to the origin
    v[:, 2] = v[:, 2] - z_min
        
    if not osp.exists(fp):
        os.makedirs(fp)
    
    igl.write_triangle_mesh(osp.join(fp, name), v, f)

    

# generate a banch of meshes samples from the latent space
def generate_mesh_samples(model, latent_vals: np.ndarray, mean, std, latent_dim):
    """_summary_
        generate meshes from the latent values
        save the meshes to the result folder

    Args:
        model_path (str): the path to the model file
        latent_vals (np.ndarray): of shape (n_samples,). The latent values for the first latent dimension.
    """

    # move the mean and std to the device
    mean = mean.to(device)
    std = std.to(device)


    # create a tensor of shape (n_samples, 8)
    latent_vectors = np.zeros((latent_vals.shape[0], 8))
    # set the latent dimension to be latent_vals
    latent_vectors[:, latent_dim] = latent_vals
    # convert to torch.Tensor
    latent_vectors = torch.Tensor(latent_vectors).cuda()

    # decode the latent values
    # of shape (n_samples, 10002, 3)
    v = decode_func(latent_vectors, model)
    # convert to numpy array
    v = v.detach()
    # denormalize the vertices
    v = v * std + mean

    v = v.cpu().numpy()

    # save the meshes
    for i in range(v.shape[0]):
        vertex = v[i]
        # find the lowest vertex
        z_min = vertex[:, 2].min()
        # move the lowest vertex to the origin
        vertex[:, 2] = vertex[:, 2] - z_min

        # check if the folder exists
        if not osp.exists("result/meshes/latent{}".format(latent_dim)):
            os.makedirs("result/meshes/latent{}".format(latent_dim))
        
        igl.write_triangle_mesh("result/meshes/latent{}/{:.1f}.obj".format(latent_dim, latent_vals[i]), vertex, f)

In [34]:
# latent_vals = np.array((-6, -4, -2, 0, 2, 4, 6))
# generate_mesh_samples(model=model, latent_vals=latent_vals, mean=caesar_mean, std=caesar_std, latent_dim=1)

In [35]:
# latent_vector = np.zeros(8)
# values = [-4.5, -3, -1.5, 0, 1.5, 3, 4.5]

# for i in range(len(values)):
#     latent_vector[1] = values[i]
#     latent_vector[2] = values[i]
#     latent_vector[3] = values[i]
#     generate_a_mesh(model, latent_vector, caesar_mean, caesar_std, "result/meshes/teaser", "{:.1f}.obj".format(values[i]))

In [36]:
# latent_vector = np.array((7, 0, 0, 0, 7, -7, 0, 0))
# generate_a_mesh(model=model, latent_vector=latent_vector, mean=caesar_mean, std=caesar_std, fp="result/meshes/extreme", name="reason_L.obj")

# latent_vector = np.array((-7, 0, 0, 0, -7, 7, 0, 0))
# generate_a_mesh(model=model, latent_vector=latent_vector, mean=caesar_mean, std=caesar_std, fp="result/meshes/extreme", name="reason_H.obj")

# latent_vector = np.array((-7, 0, 0, 0, 7, -7, 0, 0))
# generate_a_mesh(model=model, latent_vector=latent_vector, mean=caesar_mean, std=caesar_std, fp="result/meshes/extreme", name="extreme_H.obj")

# latent_vector = np.array((7, 0, 0, 0, -7, 7, 0, 0))
# generate_a_mesh(model=model, latent_vector=latent_vector, mean=caesar_mean, std=caesar_std, fp="result/meshes/extreme", name="extreme_L.obj")